# Clear a directory

In [ ]:
'''
import os

dir = "sample_data"
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))
'''

'\nimport os\n\ndir = "sample_data"\nfor f in os.listdir(dir):\n    os.remove(os.path.join(dir, f))\n'

# Create folders

In [ ]:
def create_path(subdir):
  if not os.path.exists(subdir):
    os.makedirs(subdir)

create_path('data_training')
create_path('data_metrics')

# Unzip files

In [ ]:
import zipfile

num_zips = 5

for i in range(num_zips):
  with zipfile.ZipFile("sample_data/training_data" + "_{}.zip".format(i), 'r') as zip_ref:
    zip_ref.extractall("sample_data/training" + "_{}".format(i))

In [ ]:
for i in range(5):
  with zipfile.ZipFile("sample_data/metrics_data" + "_{}.zip".format(i), 'r') as zip_ref_m:
    zip_ref_m.extractall("sample_data/metrics" + "_{}".format(i))

# Treat metrics data

## Read file content

In [ ]:
import json

num_samples = 0
seq_len = 0
num_instances_1 = []
num_instances_2 = []
num_instances_3 = []

for i in range(num_zips):
  metrics_path = "sample_data/metrics" + "_{}".format(i) + "/data_metrics/"
  for j in range(3):
    metrics_file = metrics_path +  "metrics_" + "epoch_{}.json".format(j)
    if j == 0:
      with open(metrics_file) as json_file: 
        metrics = json.load(json_file)
        num_samples = metrics['num_training_examples']
        seq_len = metrics['max_seq_len']
        num_instances_1.append(num_samples)
        print(num_instances_1)
    elif j == 1:
      with open(metrics_file) as json_file: 
        metrics = json.load(json_file)
        num_samples = metrics['num_training_examples']
        seq_len = metrics['max_seq_len']
        num_instances_2.append(num_samples)
        print(num_instances_2)
    elif j == 2:
      with open(metrics_file) as json_file: 
        metrics = json.load(json_file)
        num_samples = metrics['num_training_examples']
        seq_len = metrics['max_seq_len']
        num_instances_3.append(num_samples)
        print(num_instances_3) 

[6783]
[6844]
[7230]
[6783, 7535]
[6844, 7680]
[7230, 7810]
[6783, 7535, 7781]
[6844, 7680, 7824]
[7230, 7810, 7950]
[6783, 7535, 7781, 24455]
[6844, 7680, 7824, 24049]
[7230, 7810, 7950, 24460]
[6783, 7535, 7781, 24455, 6510]
[6844, 7680, 7824, 24049, 6559]
[7230, 7810, 7950, 24460, 7210]


## Convert to np and sum

In [ ]:
import numpy as np

np_array_1 = np.array(num_instances_1)
instances_1 = np.sum(np_array_1, dtype=np.int32)

np_array_2 = np.array(num_instances_2)
instances_2 = np.sum(np_array_2, dtype=np.int32)

np_array_3 = np.array(num_instances_3)
instances_3 = np.sum(np_array_3, dtype=np.int32)

print(instances_1, instances_2, instances_3)

53064 52956 54660


In [ ]:
instances = []
instances.append(instances_1)
instances.append(instances_2)
instances.append(instances_3)

for i in range(3):
  print(instances[i])

53064
52956
54660


## Write new metrics file 

In [ ]:
for i in range(3):
  metrics_file = "data_metrics/" + "metrics_" + "epoch_{}.json".format(i)
  f = open(metrics_file, "x")

  with open(metrics_file, 'w') as metrics_file:
    metrics = {
                "num_training_examples": int(instances[i]),
                "max_seq_len": seq_len
              }
    print(metrics)
    metrics_file.write(json.dumps(metrics))

{'num_training_examples': 53064, 'max_seq_len': 256}
{'num_training_examples': 52956, 'max_seq_len': 256}
{'num_training_examples': 54660, 'max_seq_len': 256}


# Treat text data

## Read data

In [ ]:
text_1 = []
text_2 = []
text_3 = []

for i in range(num_zips):
  data_path = "sample_data/training" + "_{}".format(i) + "/data_training/"
  for j in range(3):
    data_file = data_path +  "file_epoch_{}.json".format(j)
    if j == 0:
      with open(data_file) as json_file:
        for i, line in enumerate(json_file):
          line = line.strip()
          example = json.loads(line)
          text_1.append(example)
    elif j == 1:
      with open(data_file) as json_file: 
        for i, line in enumerate(json_file):
          line = line.strip()
          example = json.loads(line)
          text_2.append(example)
    elif j == 2:
      with open(data_file) as json_file: 
        for i, line in enumerate(json_file):
          line = line.strip()
          example = json.loads(line)
          text_3.append(example)

print(len(text_1), len(text_2), len(text_3))

53064 52956 54660


In [ ]:
array = np.array([53064, 52956, 54660])
sum = np.sum(array)
sum

160680

## Write data

In [ ]:
for i in range(3):
  data_file = "data_training/" + "file_" + "epoch_{}.json".format(i)
  f = open(data_file, "x")
  
  if i == 0:    
    with open(data_file, 'w') as data_file:
      doc_instances = [json.dumps(instance, ensure_ascii=False) for instance in text_1]      
      for instance in doc_instances:
        data_file.write(instance + '\n')
  elif i == 1:
    with open(data_file, 'w') as data_file:
      doc_instances = [json.dumps(instance, ensure_ascii=False) for instance in text_2]      
      for instance in doc_instances:
        data_file.write(instance + '\n')
  elif i == 2:
    with open(data_file, 'w') as data_file:
      doc_instances = [json.dumps(instance, ensure_ascii=False) for instance in text_3]      
      for instance in doc_instances:
        data_file.write(instance + '\n')

# Make final zips

In [ ]:
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

zipf = zipfile.ZipFile('training_data.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('data_training', zipf)

zipf.close()

In [ ]:
zipf_metrics = zipfile.ZipFile('metrics_data.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('data_metrics', zipf_metrics)

zipf_metrics.close()